In [1]:
import numpy as np
from math import inf, sqrt
import random

In [14]:
class Tictactoe:
    _board_size_to_connect = {3: 3, 5: 4, 7: 4}
    _id_to_player = {-1: 'X', 0: ' ', 1: 'O'}
    _ia_mode = ["alphabeta", "minimax", "random", ""]

    def __init__(self):
        return

    def _update_fboard(self):
        self.fboard = np.fliplr(self.board)

    def _update_tboard(self):
        self.tboard = self.board.transpose()

    def _set_board(self):
        self.board = np.array(self.board_size * [self.board_size * [0]])

    def _reset_game(self):
        """
        Reset all class attribute to a new game
        """
        self.connect = self._board_size_to_connect[self.board_size]
        self._set_board()
        self._update_fboard()
        self.turn = 1
        self.win = 0

    def _play(self, player, x, y):
        """
        Play a move on the class board at coordinates: (x, y) for player: player
        Return True if the move was succesfully played
        Return False and print an error if the move is invalid (out of the board or cell not empty)
        """
        try:
            if self.board[x][y] == 0:
                self.board[x][y] = player
                return True
            else:
                print(f"Invalid play to coordinates ({x}, {y}) : Cell is not empty")
                return False
        except:
            print(f"Invalid play to coordinates ({x}, {y}) : (the coordinates might be out of bound)")
            return False

    def _update_win(self, s):
        if abs(s) == self.connect:
            self.win = s / self.connect

    def _check_win(self):
        """
        Check the class board for a win and update the win attribute
        """
        self._update_fboard()
        self._update_tboard()
        for i in range(self.board_size):
            for j in range(self.board_size - self.connect + 1):
                self._update_win(self.board[i][j:j + self.connect].sum())
                self._update_win(self.tboard[i][j:j + self.connect].sum())
        for i in range(self.board_size - self.connect + 1):
            for j in range(self.board_size - i - self.connect + 1):
                self._update_win(self.board.diagonal(i)[j:j + self.connect].sum())
                self._update_win(self.fboard.diagonal(i)[j:j + self.connect].sum())
                if i != 0:
                    self._update_win(self.board.diagonal(-i)[j:j + self.connect].sum())
                    self._update_win(self.fboard.diagonal(-i)[j:j + self.connect].sum())

    def new_game(self):
        """
        Start a new game and play it
        Three possible board sizes : [3, 5, 7]
        Four game modes : {0: PvP, 1: PvIA, 2: IAvP, 3: IAvIA}
        Three IAs : [random, minimax, alphabeta]
        Possible to chose the depth of the search for minimax and alphabeta
        """
        while(True):
            board_size = input("Choose a board size in [3, 5, 7] : ")
            try:
                board_size = int(board_size)
            except:
                pass
            if (board_size in self._board_size_to_connect.keys()):
                self.board_size = board_size
                break
            print(f"Invalid board size : {board_size}")
        
        while(True):
            mode = input("Choose a game mode {0: PvP, 1: PvIA, 2: IAvP, 3: IAvIA} : ")
            try:
                mode = int(mode)
            except:
                pass
            if (mode in [0, 1, 2, 3]):
                self.mode = mode
                break
            print(f"Invalid game mode : {mode}")

        if self.mode == 1 or self.mode == 3:
            while(True):
                ia1 = input("Choose IA mode for player X in [random, minimax, alphabeta] : ")
                try:
                    ia1 = int(ia1)
                except:
                    pass
                if (ia1 in self._ia_mode):
                    self.ia1 = ia1
                    if self.ia1 == "minimax" or "alphabeta":
                        while(True):
                            depth_ia1 = input("Choose a depth for ia1 : ")
                            try:
                                depth_ia1 = int(depth_ia1)
                                self.depth_ia1 = depth_ia1
                                break
                            except:
                                print(f"Invalid depth for ia1 : {depth_ia1}")
                                pass
                    break
                print(f"Invalid IA mode : {ia1}")

        if self.mode == 2 or self.mode == 3:
            while(True):
                ia2 = input("Choose IA mode for player O in [random, minimax, alphabeta] : ")
                try:
                    ia2 = int(ia2)
                except:
                    pass
                if (ia2 in self._ia_mode):
                    self.ia2 = ia2
                    if self.ia2 == "minimax" or "alphabeta":
                        while(True):
                            depth_ia2 = input("Choose a depth for ia2 : ")
                            try:
                                depth_ia2 = int(depth_ia2)
                                self.depth_ia2 = depth_ia2
                                break
                            except:
                                print(f"Invalid depth for ia2 : {depth_ia2}")
                                pass
                    break
                print(f"Invalid IA mode : {ia2}")

        self._reset_game()
        # Game loop
        while(len(self.get_empty_cells(self.board)) > 0):
            print(self)
            if self.mode == 0 or self.mode == 1 and self.turn == 1 or self.mode == 2 and self.turn == -1:
                legal_play = False
                while (not legal_play):
                    input_coor = input(f"Player {self._id_to_player[self.turn]}, please input coordinate xy : ")
                    if len(input_coor) >= 2:
                        legal_play = self._play(self.turn, int(input_coor[0]), int(input_coor[1]))
                    else:
                        print(f"Invalid coordinates (1) : {input_coor}")
            else:
                ia = self.ia2 if self.turn == 1 else self.ia1
                if ia == "minimax":
                    depth = self.depth_ia2 if self.turn == 1 else self.depth_ia1
                    result = self._minimax(self.board, depth, self.turn, self.turn)
                elif ia == "alphabeta":
                    depth = self.depth_ia2 if self.turn == 1 else self.depth_ia1
                    result = self._alphabeta(self.board, depth, -inf, +inf, self.turn, self.turn)
                else:
                    result = self._random(self.board)
                self._play(self.turn, result[0], result[1])
                print(f"Player {self._id_to_player[self.turn]} played at ({result[0]}, {result[1]})")
            self.turn = -self.turn
            self._check_win()
            if self.win != 0:
                print(self)
                print(f"Player {self._id_to_player[self.win]} won !")
                break
        if self.win == 0:
            print(self)
            print("Game ended in a draw")

    def _evaluate_win(self, board):
        """
        evaluate a board and return the value of the winning player (-1 or 1) or 0 if the game is still on or ended in a draw
        """
        board_size = len(board)
        fboard = np.fliplr(board)
        tboard = board.transpose()
        connect = self._board_size_to_connect[board_size]

        for i in range(board_size):
            for j in range(board_size - connect + 1):
                s = board[i][j:j + connect].sum()
                if abs(s) == connect:
                    return s / connect
                s = tboard[i][j:j + connect].sum()
                if abs(s) == connect:
                    return s / connect
        for i in range(board_size - connect + 1):
            for j in range(board_size - i - connect + 1):
                s = board.diagonal(i)[j:j + connect].sum()
                if abs(s) == connect:
                    return s / connect
                s = fboard.diagonal(i)[j:j + connect].sum()
                if abs(s) == connect:
                    return s / connect
                if i != 0:
                    s = board.diagonal(-i)[j:j + connect].sum()
                    if abs(s) == connect:
                        return s / connect
                    s = fboard.diagonal(-i)[j:j + connect].sum()
                    if abs(s) == connect:
                        return s / connect
        
        return 0

    def evaluate(self, board, turn):
        """
        evaluate a board for a player turn and return a score
        The score tends towards 1 if player 1 is winning and return exactly 1 if player 1 won
        The score tends towards -1 if player -1 is winning and return exactly -1 if player -1 won
        Implement a "must play cell" heuristic where it determines unique cells where a player needs to play to not lose on the next turn. 
        If there is more of one of those cells for a player's turn or more than zero in the case of its oponent's turn, return a score of 0.99 * the turn
        Also implement a heuristic that favor moves in the center
        """
        win = self._evaluate_win(board)
        if win != 0:
            return win
        
        board_size = len(board)
        fboard = np.fliplr(board)
        tboard = board.transpose()
        connect = self._board_size_to_connect[board_size]

        # Must play heuristic
        must_play = {-1: [], 1: []}
        for i in range(board_size):
            for j in range(board_size - connect + 1):
                line = board[i][j:j + connect]
                s = line.sum()
                if abs(s) == connect - 1:
                    coor = (i, j + np.where(line == 0)[0][0])
                    if coor not in must_play[-s/abs(s)]:
                        must_play[-s/abs(s)].append(coor)
                line = tboard[i][j:j + connect]
                s = line.sum()
                if abs(s) == connect - 1:
                    coor = (j, i + np.where(line == 0)[0][0])
                    if coor not in must_play[-s/abs(s)]:
                        must_play[-s/abs(s)].append(coor)
        for i in range(board_size - connect + 1):
            for j in range(board_size - i - connect + 1):
                line = board.diagonal(i)[j:j + connect]
                s = line.sum()
                if abs(s) == connect - 1:
                    coor = (i + np.where(line == 0)[0][0], j + np.where(line == 0)[0][0])
                    if coor not in must_play[-s/abs(s)]:
                        must_play[-s/abs(s)].append(coor)
                line = fboard.diagonal(i)[j:j + connect]
                s = line.sum()
                if abs(s) == connect - 1:
                    coor = (board_size - i - 1 + np.where(line == 0)[0][0], board_size - j - 1 + np.where(line == 0)[0][0])
                    if coor not in must_play[-s/abs(s)]:
                        must_play[-s/abs(s)].append(coor)
                if i != 0:
                    line = board.diagonal(i)[j:j + connect]
                    s = line.sum()
                    if abs(s) == connect - 1:
                        coor = (i + np.where(line == 0)[0][0], j + np.where(line == 0)[0][0])
                        if coor not in must_play[-s/abs(s)]:
                            must_play[-s/abs(s)].append(coor)
                    line = fboard.diagonal(i)[j:j + connect]
                    s = line.sum()
                    if abs(s) == connect - 1:
                        coor = (board_size - i - 1 + np.where(line == 0)[0][0], board_size - j - 1 + np.where(line == 0)[0][0])
                        if coor not in must_play[-s/abs(s)]:
                            must_play[-s/abs(s)].append(coor)
        if len(must_play[-turn]) > 0:
            return 0.99 * turn
        if len(must_play[turn]) > 1:
            return 0.99 * -turn

        # Center coor heuristic
        center_coor = (board_size - 1) / 2.0
        n_cell = board_size**2
        center_score = 0
        for i in range(board_size):
            for j in range(board_size):
                center_score += board[i][j] / float(n_cell) / (1 + sqrt((center_coor - i)**2 + (center_coor - j)**2))
    
        return center_score

    def get_empty_cells(self, board):
        """
        return empty cells in a board
        """
        zero = np.where(board == 0)
        return [(i, j) for i, j in zip(zero[0], zero[1])]

    def _alphabeta(self, board, depth, alpha, beta, turn, player):
        """
        Implement minimax with alpha beta pruning for Tic Tac Toe
        """
        if player == turn:
            best = [-1, -1, -inf]
        else:
            best = [-1, -1, +inf]

        empty_cells = self.get_empty_cells(board)
        score = self.evaluate(board, turn)
        if depth == 0 or len(empty_cells) == 0 or score <= -1 or score >= 1:
            return [-1, -1, score * player]

        for cell in empty_cells:
            x, y = cell
            board[x][y] = turn
            result = self._alphabeta(board, depth - 1, alpha, beta, -turn, player)
            board[x][y] = 0
            result[0], result[1] = x, y
            if player == turn:
                if result[2] >= best[2]:
                    best = result
                alpha = max(alpha, best[2])
                if alpha > beta:
                    break
            else:
                if result[2] <= best[2]:
                    best = result
                beta = min(beta, best[2])
                if beta < alpha:
                    break
        return best

    def _minimax(self, board, depth, turn, player):
        """
        Implement minimax for Tic Tac Toe
        """
        if player == turn:
            best = [-1, -1, -inf]
        else:
            best = [-1, -1, +inf]

        empty_cells = self.get_empty_cells(board)
        score = self.evaluate(board, turn)
        if depth == 0 or len(empty_cells) == 0 or score <= -1 or score >= 1:
            return [-1, -1, score * player]

        for cell in empty_cells:
            x, y = cell
            board[x][y] = turn
            result = self._minimax(board, depth - 1, -turn, player)
            board[x][y] = 0
            result[0], result[1] = x, y

            if turn == player:
                if result[2] > best[2]:
                    best = result
            else:
                if result[2] < best[2]:
                    best = result

        return best

    def _random(self, board):
        """
        Implement random IA for Tic Tac Toe
        """
        return random.choice(self.get_empty_cells(board))

    def _update_nice_board(self):
        """
        Update the nice board representation
        """
        self.nice_board = np.array([[self._id_to_player[j] for j in row] for row in self.board])
    
    def __repr__(self):
        """
        Print the nice board
        """
        self._update_nice_board()
        return str(self.nice_board)

In [3]:
game = Tictactoe()

In [12]:
game.new_game()

Choose a board size in [3, 5, 7] : 5
Choose a game mode (0: PvP, 1: PvIA, 2: IAvP, 3: IAvIA) : 3
Choose IA mode for player X in [random, minimax, alphabeta] : minimax
Choose a depth for ia1 : 3
Choose IA mode for player O in [random, minimax, alphabeta] : alphabeta
Choose a depth for ia2 : 4
[[' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ']]
Player O played at (2, 2)
[[' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ']
 [' ' ' ' 'O' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ']]
Player X played at (1, 2)
[[' ' ' ' ' ' ' ' ' ']
 [' ' ' ' 'X' ' ' ' ']
 [' ' ' ' 'O' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ']]
Player O played at (2, 1)
[[' ' ' ' ' ' ' ' ' ']
 [' ' ' ' 'X' ' ' ' ']
 [' ' 'O' 'O' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ']]
Player X played at (2, 3)
[[' ' ' ' ' ' ' ' ' ']
 [' ' ' ' 'X' ' ' ' ']
 [' ' 'O' 'O' 'X' ' ']
 [' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ']]
Player O played at (1, 